# SPAM CLASSIFICATION
[Dataset: 	20050311_spam_2.tar.bz2	2005-03-11 23:55 ](https://spamassassin.apache.org/old/publiccorpus/)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Data Extraction

### Reduntant

In [36]:
# """import tarfile
# def ExtractDataFrom_tarbz2(Filepath):
#   try:
#     with tarfile.open(Filepath, "r:bz2") as tar:
#       tar.extractall(path="") # current directory
#       return "File Extracted Successfully"
#   except Exception as e:

# ExtractDataFrom_tarbz2("20050311_spam_2.tar.bz2")"""

# """import os
# import email
# import csv

# email_data = []

# def get_single_email_data(message):
#       subject = message["Subject"] #print("Subject:", message["Subject"])
#       body = message.get_payload() #print("Body:", body)
#       for part in message.walk():
#         if part.get_content_maintype() == "application" and part.get("Content-Disposition"):
#           filename = part.get_filename()
#       sender = message["From"]
#       recipients_to = message["To"]
#       recipients_cc = message["Cc"]
#       recipients_bcc = message["Bcc"]
#       date = message["Date"]
#       try:
#         return {
#         "Attachment": filename,
#         "Date": date,
#         "Subject": subject,
#         "Sender": sender,
#         "Recipients_To": recipients_to,
#         "Recipients_Cc": recipients_cc,
#         "Recipients_Bcc": recipients_bcc,
#         "Body": body
#     }
#       except:
#         return {
#           "Attachment": "NO ATTACHMENTS",
#           "Date": date,
#           "Subject": subject,
#           "Sender": sender,
#           "Recipients_To": recipients_to,
#           "Recipients_Cc": recipients_cc,
#           "Recipients_Bcc": recipients_bcc,
#           "Body": body
#       }



# for filename in os.listdir("/content/spam_2"):
#   i = 1
#   filepath = os.path.join("/content/spam_2", filename)
#   with open(filepath, "r", encoding="latin1") as file:
#       message = email.message_from_file(file)
#       email_data.append(get_single_email_data(message))
#       #headers = message.items()
#       #print(headers)

#       """

# """csv_file_path = "/content/email_data.csv"
# df = pd.DataFrame(email_data)
# df.to_csv(csv_file_path, index=False)"""


### .

In [40]:
import tarfile
from pathlib import Path
import urllib.request
def fetch_spam_data():   #Retrieves the data from Spam Assasin and returns the paths it was downloaded to
    spam_root = "http://spamassassin.apache.org/old/publiccorpus/"
    ham_url = spam_root + "20030228_easy_ham.tar.bz2"
    spam_url = spam_root + "20030228_spam.tar.bz2"

    spam_path = Path() / "datasets" / "spam"
    spam_path.mkdir(parents=True, exist_ok=True)
    for dir_name, tar_name, url in (("easy_ham", "ham", ham_url),
                                    ("spam", "spam", spam_url)):
        if not (spam_path / dir_name).is_dir():
            path = (spam_path / tar_name).with_suffix(".tar.bz2")
            print("Downloading", path)
            urllib.request.urlretrieve(url, path)
            tar_bz2_file = tarfile.open(path)
            tar_bz2_file.extractall(path=spam_path)
            tar_bz2_file.close()
    return [spam_path / dir_name for dir_name in ("easy_ham", "spam")]

In [41]:
ham_dir, spam_dir = fetch_spam_data()

In [42]:
ham_filenames = [f for f in sorted(ham_dir.iterdir()) if len(f.name) > 20]
spam_filenames = [f for f in sorted(spam_dir.iterdir()) if len(f.name) > 20]
len(ham_filenames), len(spam_filenames)

(2500, 500)

In [43]:
import email
import email.policy

def load_email(filepath):
  with open(filepath, "rb") as f:
    return email.parser.BytesParser(policy=email.policy.default).parse(f)


In [44]:
ham_emails = [load_email(filepath) for filepath in ham_filenames]
spam_emails = [load_email(filepath) for filepath in spam_filenames]

In [45]:
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [46]:
print(spam_emails[6].get_content().strip())

Help wanted.  We are a 14 year old fortune 500 company, that is
growing at a tremendous rate.  We are looking for individuals who
want to work from home.

This is an opportunity to make an excellent income.  No experience
is required.  We will train you.

So if you are looking to be employed from home with a career that has
vast opportunities, then go:

http://www.basetel.com/wealthnow

We are looking for energetic and self motivated people.  If that is you
than click on the link and fill out the form, and one of our
employement specialist will contact you.

To be removed from our link simple go to:

http://www.basetel.com/remove.html


4139vOLW7-758DoDY1425FRhM1-764SMFc8513fCsLl40


In [ ]:
def get_email_structure(email):
  if isInstance(email, str):
    return email
  payload = email.get_payload()
  if isinstance(payload, list):
    multipart = ", ".join([get_email_structure(sub_email) for sub_email in payload])
    return f"multipart({multipart})"
  else:
    return email.get_content_type()

## DATA PREPROCESSING

### Redundant

In [34]:
datafile = "/content/email_data.csv"
data = pd.read_csv(datafile)
data.columns

Index(['Attachment', 'Date', 'Subject', 'Sender', 'Recipients_To',
       'Recipients_Cc', 'Recipients_Bcc', 'Body'],
      dtype='object')

### .